## Práctica 16: Clasificación de imágenes con la API de Custom Vision (Microsoft Azure)

### Objetivo de la práctica

El objetivo de esta práctica es crear un modelo de **clasificación de imágenes** utilizando el servicio **Custom Vision** de Microsoft Azure, cubriendo todas las fases del proceso:

- Creación y configuración del proyecto en la plataforma.
- Carga de imágenes y etiquetado por clases.
- Entrenamiento automático del modelo.
- Evaluación con nuevas imágenes.
- Inferencia mediante la **API REST** usando Python.
- Visualización de los resultados.


## 1. Preparación del entorno

In [44]:
!pip install azure-cognitiveservices-vision-customvision

## 2. Instalación de librerías y configuración de la API

Importamos las librerías necesarias y configuramos la conexión con nuestro proyecto de Custom Vision. Es importante mantener seguras las claves API, evitando exponerlas en repositorios públicos.


In [46]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import os

# Aquí vamos a usar mis datos
ENDPOINT = "https://westeurope.api.cognitive.microsoft.com" # endpoint aqui
PREDICTION_KEY = "60c15c78ac234257bace0c5a38103bb2"  # prediction key aqui
PROJECT_ID = "42fdc420-14c8-4a46-9410-8c8ec835bef5"  # project ID aqui
PUBLISH_ITERATION_NAME = "Iteration1" # no le he cambiado el nombre

# Autenticacion
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)


## 3. Clasificación de nuevas imágenes

Para comprobar el rendimiento del modelo, utilizamos imágenes que **no se usaron en el entrenamiento**. Estas se procesan desde una carpeta local y se clasifican mediante la API de predicción.


In [48]:
# Carpeta con imagenes para probar
test_images_folder =  "./test"  # Cabiar si cambia la ruta de la carpeta

# Leer y clasificar cada imagen
for image_file in os.listdir(test_images_folder):
    if image_file.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(test_images_folder, image_file)
        with open(image_path, "rb") as image_data:
            results = predictor.classify_image(
                PROJECT_ID, PUBLISH_ITERATION_NAME, image_data.read()
            )
            
            # Mostrar resultados
            print(f"\nResultados para imagen: {image_file}")
            for prediction in results.predictions:
                print(f"\n{prediction.tag_name}: {prediction.probability * 100:.2f}% de confianza")



Resultados para imagen: 1.png

Ardilla: 99.99% de confianza

Loro: 0.01% de confianza

Tortuga: 0.00% de confianza

Resultados para imagen: 2.png

Ardilla: 88.23% de confianza

Loro: 11.77% de confianza

Tortuga: 0.00% de confianza

Resultados para imagen: 3.png

Ardilla: 100.00% de confianza

Loro: 0.00% de confianza

Tortuga: 0.00% de confianza

Resultados para imagen: 4.png

Tortuga: 99.94% de confianza

Loro: 0.06% de confianza

Ardilla: 0.00% de confianza

Resultados para imagen: 5.png

Tortuga: 99.94% de confianza

Loro: 0.06% de confianza

Ardilla: 0.00% de confianza

Resultados para imagen: 6.png

Tortuga: 99.67% de confianza

Loro: 0.31% de confianza

Ardilla: 0.02% de confianza

Resultados para imagen: 7.png

Loro: 100.00% de confianza

Ardilla: 0.00% de confianza

Tortuga: 0.00% de confianza

Resultados para imagen: 8.png

Loro: 100.00% de confianza

Tortuga: 0.00% de confianza

Ardilla: 0.00% de confianza

Resultados para imagen: 9.png

Loro: 100.00% de confianza

Ardilla:

## 4. Resultados de las predicciones

El modelo demostró un rendimiento muy sólido, diferenciando correctamente entre clases como **Ardilla**, **Tortuga** y **Loro**.


## 5. Evaluación en la plataforma Custom Vision

### Vista general del proyecto : Preparación del dataset en Custom Vision

En la plataforma Azure Custom Vision, se creó un nuevo proyecto de clasificación multicategoría. Se subieron un total de 90 imágenes, divididas en tres clases: ardilla (30 imágenes), tortuga (30 imágenes) y loro (30 imágenes).
Cada imagen fue etiquetada manualmente con su categoría correspondiente, asegurando que el conjunto de datos estuviera equilibrado en número de ejemplos por clase. Este paso es crucial para evitar que el modelo se sesgue hacia una clase en particular.

La plataforma permite visualizar rápidamente el número de imágenes por clase, facilitando el control de calidad del dataset antes del entrenamiento.

<p align="center">
  <img src="imagenes_customvision/proyecto_customvision.png" width="1000"/>
</p>

Aquí se muestra la interfaz del proyecto en Custom Vision


### Resultados del entrenamiento del modelo : Entrenamiento del modelo en Custom Vision

Una vez preparado el conjunto de datos, se inició el entrenamiento automático del modelo seleccionando el tipo de clasificación con una sola etiqueta por imagen. El modelo fue entrenado utilizando los algoritmos internos de Custom Vision, que dividen automáticamente las imágenes en subconjuntos de entrenamiento y validación (80/20).

Al finalizar el entrenamiento, la plataforma mostró métricas clave para evaluar el rendimiento del modelo:

Precisión (Precision): Proporción de predicciones correctas entre las imágenes que el modelo identificó como una clase determinada.

Recall: Proporción de imágenes correctamente identificadas entre todas las imágenes reales de esa clase.

Average Precision (AP): Métrica que resume el área bajo la curva precisión-recall, útil para evaluar el rendimiento general del modelo.

Los resultados obtenidos mostraron un buen equilibrio entre precisión y recall en las tres clases, lo cual indica que el modelo es capaz de distinguir correctamente entre las imágenes de ardillas, tortugas y loros.
 
<p align="center">
  <img src="imagenes_customvision/entrenamiento.png" width="700"/>
</p>

El modelo obtuvo una precisión del 100%, un resultado muy elevado que probablemente se debe a que las imágenes utilizadas son visualmente simples y bien diferenciadas. Si se hubieran utilizado imágenes más complejas —por ejemplo, con variaciones de ángulo, color o iluminación—, es probable que la precisión hubiera disminuido. Sin embargo, esto habría dado lugar a un modelo más robusto y realista, con mejor capacidad de generalización ante situaciones del mundo real.


### Predicción rápida (Quick Test) : Resultados del Quick Test en Custom Vision

Para validar el comportamiento del modelo, se utilizó la funcionalidad de Quick Test en la plataforma, subiendo un total de 9 imágenes nuevas (3 por clase) que no habían sido utilizadas durante el entrenamiento.

Cada imagen fue evaluada de forma individual y el modelo devolvió:

La clase predicha.

La probabilidad asociada a esa predicción.

En algunos casos, un porcentaje de confianza bajo si la imagen era ambigua o poco clara, lo cual es un indicador útil sobre la incertidumbre del modelo.

En general, el modelo identificó correctamente la mayoría de las imágenes, mostrando una buena capacidad de generalización. En las pocas imágenes donde hubo duda, el modelo aún fue capaz de identificar la clase más probable con una puntuación aceptable.

A continuación se muestran las 9 imágenes utilizadas en la prueba rápida del modelo entrenado en Custom Vision.

<p align="center">
  <img src="imagenes_customvision/quick_test_1.png" width="350"/>
  <img src="imagenes_customvision/quick_test_2.png" width="350"/>
</p>
<p align="center">
  <img src="imagenes_customvision/quick_test_3.png" width="350"/>
  <img src="imagenes_customvision/quick_test_4.png" width="350"/>
</p>
<p align="center">
  <img src="imagenes_customvision/quick_test_5.png" width="350"/>
  <img src="imagenes_customvision/quick_test_6.png" width="350"/>
</p>
<p align="center">
  <img src="imagenes_customvision/quick_test_7.png" width="350"/>
  <img src="imagenes_customvision/quick_test_8.png" width="350"/>
</p>
<p align="center">
  <img src="imagenes_customvision/quick_test_9.png" width="350"/>
</p>


En esta prueba rápida, el modelo identificó correctamente el contenido de las imágenes subidas.

## 6. Conclusiones

✅ El modelo fue entrenado con éxito usando Custom Vision en Azure.  
✅ Se logró una alta precisión en la clasificación de imágenes no vistas.  
✅ La integración mediante la API REST es sencilla y efectiva para despliegue real.

La plataforma Custom Vision de Microsoft Azure ha demostrado ser una herramienta accesible y potente para el desarrollo rápido de modelos de clasificación de imágenes. Su interfaz intuitiva permite cargar y etiquetar imágenes de forma sencilla, facilitando así el entrenamiento de modelos sin necesidad de conocimientos avanzados de machine learning.

En este proyecto, se ha entrenado un modelo con 90 imágenes organizadas en tres clases (ardilla, tortuga y loro), obteniendo una precisión del 100%. Aunque este resultado es excelente en términos numéricos, es importante destacar que se ha logrado en un entorno controlado con imágenes sencillas y sin ruido. Para aplicaciones más realistas, se recomienda aumentar la complejidad del conjunto de datos y aplicar técnicas de aumento de datos para mejorar la capacidad del modelo de generalizar.

La opción de Quick Test ha permitido validar rápidamente el rendimiento del modelo frente a nuevas imágenes, demostrando una correcta clasificación en los tres casos. No obstante, para una evaluación más rigurosa se debería utilizar un conjunto de prueba independiente y más diverso.

Finalmente, el uso del script en Python para conectarse al modelo publicado, enviar imágenes y recibir predicciones ha sido fundamental para validar el despliegue del modelo de manera programática. Esta integración permite automatizar procesos, escalar el uso del modelo en aplicaciones reales y comprender el flujo completo desde la creación hasta el consumo del modelo.